# 소방청 - 타지역 검색

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome()

In [4]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5605672&myY=126.9433486&searchKeyword='
driver.get(url)

In [ ]:
#입력창에 광주를 입력하고 버튼을 누르는 동작
driver.find_element(By.ID 'searchKeyword').send_keys('서울')
driver.find_element(By.ID 'fsSearchBtn').click()

In [7]:
#이동후에 Beautiful을 사용해 정보를 획득
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [11]:
li = lis[0]
name = li.select_one('.title').get_text().strip()
name

'광산소방서_광주-평동-119 안전센터'

In [13]:
addr = li.find('address').string.strip()
addr

'광주광역시 광산구 월전동 1251'

In [15]:
tel = li.select_one('.tel').get_text().strip()
tel

'전화공사중'

In [16]:
lines = []
for il in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.find('address').string.strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append([name, addr, tel])

## 두번째 페이지로 이동

In [17]:
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()

In [8]:
#이동후에 Beautiful을 사용해 정보를 획득
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [19]:
# 다음페이지로 이동하는 다른방식
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

## 전체

In [21]:
# 검색결과 건수를 가져오기
strong = soup.select('.stations-local-after > p > strong')
strong[1].get_text()

'35건'

In [22]:
count = int(strong[1].get_text().replace('건', '').strip())
count

35

In [23]:
import math
count = math.ceil(count/10)
count

4

# 전체 페이지

In [9]:
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [11]:
import math
strong = soup.select('.stations-local-after > p > strong')
count = int(strong[1].get_text().replace('건', '').strip())
count = math.ceil(count/10)

In [12]:
lines = []
for i in range(count):
    if i >= 1 and i%2 == 0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(2)
    if i >= 1 and i%2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').string.strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name, addr, tel])    

In [13]:
len(lines)

100

In [14]:
df = pd.DataFrame(lines, columns = ['관서명', '주소', '전화번호'])
df

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
...,...,...,...
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119


In [15]:
len(df['관서명'].unique())

100

In [16]:
df.to_csv('서울소방서.csv', index=False)

##  카카오 api 사용해서 위도경도 가져오기

In [28]:
import requests
from urllib.parse import quote

In [29]:
with open ('kakao.txt') as file:
    kakao_key = file.read()

In [30]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization':f'KakaoAK {kakao_key}'}

In [31]:
# 반복문을 통해서 모든 소방청 위도 경도 좌표 구하기
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    try: 
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

은평소방서-수색-119 안전센터


In [32]:
df[df.관서명.isin(['은평소방서-수색-119 안전센터'])].주소

30    서울특별시 은평구 수색로 294 (수색동)
Name: 주소, dtype: object

In [35]:
url = f'{local_url}?query={quote("서울특별시 은평구 수색로 294")}'
result = requests.get(url, headers=header).json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [36]:
df[df.관서명.isin(['은평소방서'])].주소

5    서울특별시 은평구 통일로 962(진관동)
Name: 주소, dtype: object

In [38]:
df.주소[30] = '서울특별시 은평구 통일로 962'

In [39]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    try: 
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

In [40]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head(2)

,관서명,주소,전화번호,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,37.494672,126.917719
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,37.573205,126.935996


In [41]:
import folium